In [1]:
import pandas as pd
import numpy as np
import string
import os
import smart_open
import random
import datetime
import json
import re
import heapq
import pickle
import sys

from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from collections import Counter


from gensim.models import Phrases
import gensim
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')



# Understand the table

In [2]:
post_df = pd.read_csv('all_posts.csv', sep = "\t")
Q_df = post_df.loc[post_df['PostTypeId'] == 1][['Id', 'Title','Body']]
A_df =  post_df.loc[post_df['PostTypeId'] != 1][['Id', 'Body']]


In [3]:
original_q_posts = Q_df['Body']
original_a_posts = A_df['Body']
original_q_titles = Q_df['Title']


In [4]:
if not os.path.exists('./text/'):
            os.makedirs('./text/')
with open('./text/questions_body.txt', 'w') as thefile:
    for post in original_q_posts:
        post = str(post).replace('\n', ' ')
        post = re.sub(r"\$.*\$", "", post)
        thefile.write("%s\n" % post)

with open('./text/answers_body.txt', 'w') as thefile:
    for post in original_a_posts:
        post = str(post).replace('\n', ' ')
        post = re.sub(r"\$.*\$", "", post)
        thefile.write("%s\n" % post)

with open('./text/questions_title.txt', 'w') as thefile:
    for i, post in enumerate(original_q_titles):
        post = str(post).replace('\n', "") 
        post = re.sub(r"\$.*\$", "", post)
        thefile.write("%s\n" % post)

In [5]:
with open('./text/all_posts_body.txt','w') as thefile:
    for i, post in enumerate(original_q_posts):
        post = str(post).replace('\n', "") 
        post = re.sub(r"\$.*\$", "", post)
        thefile.write("%s\n" % post)
    for i, post in enumerate(original_a_posts):
        post = str(post).replace('\n', "") 
        post = re.sub(r"\$.*\$", "", post)
        thefile.write("%s\n" % post)

In [6]:
i = 0
with open('./text/all_posts_body.txt','r') as thefile:
    for line in thefile:
        i += 1
print(i==post_df.shape[0])
print(post_df.shape[0])

True
215962


### data preprocessing


In [7]:
# def is_number(s):
#     try:
#         float(s)
#         return True
#     except ValueError:
#         return False

In [8]:
# def show_work_status(singleCount, totalCount, currentCount = 0):
#     currentCount += singleCount
#     percentage = currentCount/totalCount *100
#     status = ">" * int(percentage) +  " " * (100-int(percentage))
#     sys.stdout.write('\rStatus:[{0}] {1:.2f}%'.format(status, percentage))
#     sys.stdout.flush()
#     if percentage >= 100:
#     	print('\n')

In [9]:
# translation = str.maketrans(string.punctuation,' '*len(string.punctuation))

In [10]:
# preprocess: 
# 1. lower case
# 2. tokenize
# 3. add to vocab (include bigrams)


In [11]:
def preprocess(fname):
    sentences = []
    bigram = Phrases(min_count=5)
    with open(fname, "r") as f:
        for line in f:
            sentence = gensim.utils.simple_preprocess(line) 
#             This lowercases, tokenizes, de-accents (optional). –  
#             the output are final tokens = unicode strings, that won’t be processed any further.
#             sentence is a list of words. only lowercases and de-accents, no use lemmatize or stopwords.
            sentences.append(sentence)
            bigram.add_vocab([sentence])
    if not os.path.exists('../SO_bigram/'):
            os.makedirs('../SO_bigram/')
    with open("../SO_bigram/SO_bigram.pkl", 'wb') as f:
        pickle.dump(bigram,f)
    return bigram, sentences


In [12]:
all_bigram, all_sentences = preprocess('./text/all_posts_body.txt')

In [13]:
with open("../SO_bigram/SO_bigram.pkl", 'rb') as f:
     new_bigram =  pickle.load(f)
len(new_bigram.vocab.keys())

3386232

In [14]:
print(len(all_bigram.vocab))
print(list(all_bigram.vocab.keys())[0:50])
print(list(all_bigram[all_sentences])[:5])


3386232
[b'how', b'how_should', b'should', b'should_elicit', b'elicit', b'elicit_prior', b'prior', b'prior_distributions', b'distributions', b'distributions_from', b'from', b'from_experts', b'experts', b'experts_when', b'when', b'when_fitting', b'fitting', b'fitting_bayesian', b'bayesian', b'bayesian_model', b'model', b'in', b'in_many', b'many', b'many_different', b'different', b'different_statistical', b'statistical', b'statistical_methods', b'methods', b'methods_there', b'there', b'there_is', b'is', b'is_an', b'an', b'an_assumption', b'assumption', b'assumption_of', b'of', b'of_normality', b'normality', b'normality_what', b'what', b'what_is', b'is_normality', b'normality_and', b'and', b'and_how', b'how_do']
[['how', 'should', 'elicit', 'prior', 'distributions', 'from', 'experts', 'when', 'fitting', 'bayesian', 'model'], ['in', 'many', 'different', 'statistical', 'methods', 'there', 'is', 'an', 'assumption', 'of', 'normality', 'what', 'is', 'normality', 'and', 'how', 'do', 'know', 'if

In [15]:
def remove_stopwords(bigram, sentence):
    temp = bigram[sentence]# temp is a list of words(unigram and bigram)
    result = []
    for element in temp:
        key = element.split("_")
        if len(key) == 1:
            result.append(element)
        if len(key) > 1 and not any([word in stopwords.words("english") for word in key]):
            result.append(element)
    return result
            

In [16]:
all_sentences_nonstopword = []
for sentence in all_sentences:
    all_sentences_nonstopword.append(remove_stopwords(all_bigram, sentence))
    

In [17]:
print(len(all_sentences_nonstopword) == len(all_sentences))
print(len(all_sentences))

True
215962


In [18]:
with open('all_sentences_nonstopword.pickle', 'wb') as f:
    pickle.dump(all_sentences_nonstopword, f)

In [19]:
sum([len(all_sentences[i])- len(all_sentences_nonstopword[i]) > 10 for i in range(len(all_sentences))]) # so we filter out  a lot of words.

58999

In [20]:
# def count_bigram(bigram):
#     bigram_counter = Counter()
#     for key in bigram.vocab.keys():
#         if key not in stopwords.words("english"):
#             if len(key.decode('utf-8').split("_")) > 1 and not any([word in stopwords.words("english") for word in key.decode('utf-8').split("_")]):
#                 bigram_counter[key] = bigram.vocab[key]
#     for key, counts in bigram_counter.most_common(20):
#         print(key,counts) 

In [21]:
# count_bigram(all_bigram) # the first 20 common bigram without stopwords in the vocabulary.

In [22]:
# most of them are out of interest. So we have to move the bigram with stopword out of the vocabulary. 

In [23]:
# def vocab(bigram):
#     vocab_dict = Counter()
#     for key,value in bigram.vocab.items():
#         words = key.decode("utf-8").split("_")
#         if len(words) == 1:
#             if key.decode("utf-8") not in stopwords.words("english"):
#                 vocab_dict[key.decode('utf-8')] = value    
#         else:
#             if not any([word in stopwords.words('english') for word in words]):
#                 vocab_dict[key.decode('utf-8').replace("_", " ")] = value
#     vocab = list(vocab_dict.keys())
#     return vocab, vocab_dict

In [24]:
# title_vocab,title_vocab_dict = vocab(title_bigram)

In [25]:
# for key, counts in title_vocab_dict.most_common(200):
#     print(key, counts)

In [26]:
# # It takes a while.
# title_vocab,title_vocab_dict = vocab(title_bigram)
# question_vocab,question_vocab_dict = vocab(question_bigram)
# answer_vocab,answer_vocab_dict = vocab(answer_bigram)

In [27]:
# print(len(title_vocab))
# print(title_vocab_dict.most_common(20))

# print(len(question_vocab))
# print(question_vocab_dict.most_common(20))

# print(len(answer_vocab))
# print(answer_vocab_dict.most_common(20))



# Parameters for word2vec models:

In [28]:
# #parameters
# sizes = np.linspace(100, 200, num = 1)
# print(sizes)
# windows = np.linspace(6, 6,num = 1 )
# print(windows)
# min_counts = [2]
# print(min_counts )


 Later make it more dense.

In [29]:
# After tuning the model and find appropriate parameters, save them in the config file and read from the config file. 


with open('../../config/config.json', 'r') as f:
    data = json.load(f)
    paras = data['model_paras']
    
min_count = 1
size = paras['size']
window = paras['window']
# workers = paras['workers']
# sg = paras['sg']
# alpha = paras["alpha"]
# hs = paras["hs"]
# negative = paras["negative"]
print(min_count)
print(size)
print(window)

1
200
6


# Word2Vec Model

In [30]:
# build models

In [31]:
if not os.path.exists('./model_bigram/'):
            os.makedirs('./model_bigram/')

In [35]:
# note: sentence: a list of lists of words.
# paras is a dictionary{ para: value, para: value}

def train_word_model(**paras):
    min_count = paras['min_count']
    size = paras['size']
    window = paras['window']
    bigram_model = gensim.models.Word2Vec(all_sentences_nonstopword, min_count = min_count, size = size, window = window, workers = 4)
    with open("./model_bigram/bigram_model.pkl", 'wb') as f:
        pickle.dump(bigram_model,f)
    return bigram_model

### Training  models or loading models.

In [36]:
paras

{'min_count': 1, 'size': 200, 'window': 6}

In [37]:
paras = {'size':int(size), 'window' : int(window), "min_count" :int(min_count)}
all_post_model = train_word_model( **paras)

In [38]:
len(all_post_model.wv.vocab)

220018

In [39]:
['new_york' in all_post_model.wv.vocab, 'random_walk' in all_post_model.wv.vocab]

[True, True]

In [40]:
def glance_word(model):
    print(model)
    print('*************most similar words to \'vector\'***************')
    print(model.most_similar('vector'))
    print('\n')
    print("**************Similarity of \'probability\' and \'distribution\'******************")
    print(model.similarity('probability','distribution'))
    print('\n')
    print("**************Similarity of \'gaussian\' and \'normal\'******************")
    print(model.similarity('gaussian','normal'))
    print('\n')
    print("************** most_similar(positive=['neural_network']) ******************")
    print(model.most_similar(positive=['neural_network']))
    print('\n')
    print("************** most_similar(positive=['p_value']) ******************")
    print(model.most_similar(positive=['p_value']))
    print('\n')
    print("************** most_similar(positive=['gaussian']) ******************")
    print(model.most_similar(positive=['gaussian']))
    print('\n')  

In [41]:
glance_word(all_post_model)

Word2Vec(vocab=220018, size=200, alpha=0.025)
*************most similar words to 'vector'***************
[('vectors', 0.7079745531082153), ('feature_vector', 0.6482966542243958), ('matrix', 0.6289140582084656), ('array', 0.6275156140327454), ('scalar', 0.6212510466575623), ('element', 0.6047627925872803), ('dot_product', 0.572348415851593), ('input', 0.5614036321640015), ('vector_containing', 0.5606834292411804), ('transpose', 0.55959153175354)]


**************Similarity of 'probability' and 'distribution'******************
0.399785206023


**************Similarity of 'gaussian' and 'normal'******************
0.672543092038


************** most_similar(positive=['neural_network']) ******************
[('neural_net', 0.8596671223640442), ('network', 0.8122416138648987), ('cnn', 0.8027771711349487), ('rnn', 0.7903342247009277), ('mlp', 0.7707802057266235), ('neural_networks', 0.765505313873291), ('feed_forward', 0.7543053030967712), ('architecture', 0.748978853225708), ('ann', 0.7467960

In [ ]:
# it seems that the word vectors are good.

In [42]:
def save_vectors(model):
    word_vec_dic = {}
    vocab = model.wv.vocab.keys()
    for word in vocab:
        word_vec_dic[word] = model[word]
    return word_vec_dic 

In [43]:
# save word vectors for all models in model_word_vec_dic, model_word_vec_df and also in csv form.

if not os.path.exists('../word_vec_bigram/'):
            os.makedirs('../word_vec_bigram/')

model_word_vec_dic = save_vectors(all_post_model)
model_word_vec_df = pd.DataFrame(model_word_vec_dic).T
model_word_vec_df.to_csv('../word_vec_bigram/all_posts_word_vec.csv')


In [44]:
vocab = list(model_word_vec_df.index)

In [45]:
print(model_word_vec_df.head())
print(model_word_vec_df.tail())
print(model_word_vec_df.shape)


            0         1         2         3         4         5         6    \
aa     0.068121  0.597183  0.409198 -0.011180  0.871632  0.063611 -0.282838   
aa_    0.004616 -0.022516 -0.007492  0.037375  0.030645 -0.007611  0.027959   
aa_aa  0.085667 -0.151838 -0.004150 -0.033897  0.360972 -0.107091  0.034174   
aa_ab  0.110608  0.047761 -0.105305 -0.044673  0.613378 -0.116470  0.102508   
aa_bb  0.032411 -0.146912  0.010144  0.103219  0.579319 -0.323206  0.230181   

            7         8         9      ...          190       191       192  \
aa    -0.257215  0.300768  0.332370    ...    -1.464614  0.693200 -0.771847   
aa_   -0.005872  0.016260  0.009216    ...    -0.006271  0.023406 -0.013864   
aa_aa -0.058881  0.446170  0.248519    ...    -0.449324  0.061906 -0.296851   
aa_ab -0.114061  0.745889  0.533484    ...    -0.600632  0.143180 -0.272454   
aa_bb -0.109698  0.679535  0.395721    ...    -0.606515  0.061151 -0.241730   

            193       194       195       196     

In [46]:
print(vocab[-100:])

['об', 'образов', 'обратных', 'обучение', 'обшествоя', 'ожидаемых', 'определяют', 'опять', 'остатки', 'от', 'отклонений', 'паклин', 'последняя', 'поставленных', 'похожа', 'правило', 'предел', 'прикладная', 'приятно', 'проблема', 'процесса', 'проще', 'пуассона', 'распознавание', 'расстояние', 'расстоянии', 'рациональный', 'регрессии', 'регуляризации', 'реферативный', 'решении', 'россия', 'сrdist', 'сао', 'сил', 'случайной', 'спецификации', 'средних', 'ссср', 'стандартных', 'стьюд', 'сушку', 'так', 'тел', 'технологий', 'типичные', 'то', 'трех', 'ул', 'устойчивости', 'факторы', 'фот', 'части', 'шансов', 'этого', 'юрьев', 'التي', 'السلام', 'الله', 'المساحة', 'اليمين', 'بركاته', 'تعني', 'رحمة', 'على', 'عليكم', 'ـnote', '₁aₓ', 'ℓn', '上海', '将軍', '洪湖', '洪湖i', '洪湖松花皮蛋', 'ﬁeld', 'ﬁelds', 'ﬁlter', 'ﬁlteryields', 'ﬁnancial', 'ﬁnd', 'ﬁnding', 'ﬁnds', 'ﬁnite', 'ﬁrm', 'ﬁrms', 'ﬁrst', 'ﬁrst_outcome', 'ﬁt', 'ﬁtdistrplus', 'ﬁts', 'ﬁtted', 'ﬁtting', 'ﬁve', 'ﬁxed', 'ﬂawed', 'ﬂexible', 'ﬂinch', 'ﬂow', 'ﬂuc

From vectors of words, how to get the vectors of docs? n_similarity uses the mean of the vectors of all words in the doc. We will use the tf-idf weighted vector. 

## Getting tf_idf of all words  in the vocab in all documents as weights  for all models.
They are to be the weights of the words when we calculate the vector of docs.

In [47]:
for i in range(10):
    print(all_sentences_nonstopword[i])

['how', 'should', 'elicit', 'prior', 'distributions', 'from', 'experts', 'when', 'fitting', 'bayesian', 'model']
['in', 'many', 'different', 'statistical', 'methods', 'there', 'is', 'an', 'assumption', 'of', 'normality', 'what', 'is', 'normality', 'and', 'how', 'do', 'know', 'if', 'there', 'is', 'normality']
['what', 'are', 'some', 'valuable', 'statistical', 'analysis', 'open_source', 'projects', 'available', 'right', 'now', 'edit', 'as', 'by', 'sharpie', 'valuable', 'could', 'mean', 'helping', 'you', 'get', 'things', 'done', 'faster', 'or', 'more', 'cheaply']
['have', 'two', 'groups', 'of', 'data', 'each', 'with', 'different', 'distribution', 'of', 'multiple', 'variables', 'trying', 'to', 'determine', 'if', 'these', 'two', 'groups', 'distributions', 'are', 'different', 'in', 'statistically_significant', 'way', 'have', 'the', 'data', 'in', 'both', 'raw', 'form', 'and', 'binned', 'up', 'in', 'easier', 'to', 'discrete', 'categories', 'with', 'frequency', 'counts', 'in', 'each', 'what', '

In [48]:
corpus = []
for i in range(len(all_sentences_nonstopword)):
    temp = " ".join(all_sentences_nonstopword[i])
    corpus.append(temp)
    

In [49]:
vectorizer = TfidfVectorizer(min_df=1, vocabulary=vocab)
X = vectorizer.fit_transform(corpus)
X.shape

(215962, 220018)

In [50]:
# save the vocab from tf_idf vectorizer for future reference.
import csv
if not os.path.exists('../vocab_dic_bigram/'):
            os.makedirs('../vocab_dic_bigram/')

fname = '../vocab_dic_bigram/vocab_dict_question_answer.csv'
with open(fname, 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in vectorizer.vocabulary_.items():
        writer.writerow([key, value])

In [51]:
len(vectorizer.vocabulary_)

220018

## Get weighted vectors for all documents.

This part is used for intrinsic evaluation. We have found that the result is satisfying, though one reason maybe that the evaluation is on the training dataset itself. This part is time consuming, so if only want to get word vectors, then there is no need to run this part.

In [ ]:
# # The number of nonzero elements in a row in X should equal to the number 
# # of unique words in the corresponding doc or word2vec_question_corpus. 
# # The funciton check is used to check this.

# def check(key,i): # ith documents. doc_corpus a list of words
#     doc_corpus = word2vec_question_corpus[i]
#     df = model_word_vec_df[key]
#     related_rows = df.loc[sorted(list(set(doc_corpus).intersection(set(vocab)))), :]  
#     words_index = set(np.where(df.index.isin(set(doc_corpus).intersection(set(vocab))))[0])
#     # print(related_rows.T)
#     weights_index = []
#     ind = sorted(X[i,:].nonzero()[1])
#     for j in ind:
#         weights_index.append(j)
   
#     if len(weights_index) - len(words_index) != 0:
#         print(i)
#         print(weights_index)
#         print(words_index)
#     if related_rows.shape[0] != len(weights_index):
#         print(i)
#         print("*********")
#     return 
    

In [ ]:
def save_obj(obj, name ):
    with open('doc_vec/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name ):
    with open('doc_vec/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
# ith documents. doc_corpus a list of words 
# only do this for questions
def get_doc_weighted_vec(key,i, weighted = True): 
    doc_corpus = word2vec_question_corpus[i]
    df = model_word_vec_df[key]
    related_rows = df.loc[sorted(list(set(doc_corpus).intersection(set(vocab)))), :] 
    
    if weighted:
        weights = []
        ind = sorted(X[i,:].nonzero()[1]) # words index 
# One should check this. But it is very slow. I checked half of all question posts and they are valid. 
#         if sum([vectorizer.vocabulary_[related_rows.index[j]] != ind[j] for j in range(len(ind))]) != 0:
#             print("words position don't match")
#             return 

        weights = [X[i, ind[j]] for j in range(len(ind))]        
        weights = np.array(weights)/sum(weights) # scale so the sum of weights is one.
    else:
        # to correct: some words may appear more than once. But since we are only interested in the weighted vec, I will correct this later.
        weights = [1/related_rows.shape[0]] * related_rows.shape[0] 
 

    result = related_rows.T * weights
    return result.sum(axis = 1) # weighted sum as the doc vector.

In [ ]:
questions_vec_weighted = []
total = len(word2vec_question_corpus)
current = 0
for i in range(len(word2vec_question_corpus)):
    show_work_status(1, total, current)
    current += 1
    questions_vec_weighted.append(get_doc_weighted_vec(key,i))


print(len(questions_vec_weighted))
print(len(questions_vec_weighted[0]))

# save questions_vec_weighted
save_obj(questions_vec_weighted, "questions_weighted_vec_"+key)
questions_vec_weighted = load_obj("questions_weighted_vec_"+key)

print(len(questions_vec_weighted))
print(len(questions_vec_weighted[0]))

questions_vec_weighted_df = pd.DataFrame(questions_vec_weighted)
print(questions_vec_weighted_df.head())
print(questions_vec_weighted_df.shape)

questions_vec_weighted_df.to_csv('doc_vec/questions_weighted_vec_'+key+'.csv')

After getting the vector representations of the doc, one can use cosine similarity to get related posts given one post from the training dataset.